In [1]:
VISIT_DESCRIPTIONS_PATH = "../src/data_descriptions/visit_descriptions.csv"
REVIEWS_PATH = "../src/data_descriptions/reviews.csv"

In [2]:
import dotenv
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.schema.messages import HumanMessage, SystemMessage
dotenv.load_dotenv()

True

In [3]:
data_descriptions = pd.read_csv(VISIT_DESCRIPTIONS_PATH)
print(data_descriptions.sample()["visit_description"].values[0])

patient id: 1340 
 date of admission: 2019-01-02 
 billing amount: 15412.399249958517 
 room number: 219 
 admission type: Elective 
 discharge date: 2019-01-11 
 test results: Inconclusive 
 visit id: 1340 
 physician id: 183 
 payer id: 0 
 hospital id: 7 
 chief complaint: nan 
 treatment description: nan 
 primary diagnosis: nan 
 visit status: DISCHARGED 
 hospital name: Brown Inc 
 payer name: Medicare 
 patient name: Matthew Stevens 
 patient sex: Male 
 patient dob: 1939-07-09 00:00:00 
 patient blood type: A+ 
 physician name: Tamara Potter 
 physician dob: 1960-06-15 00:00:00 
 physician grad year: 1990-06-15 00:00:00 
 medical school: Washington University in St. Louis School of Medicine 
 salary: 282300.6422875667 



#### LLMs and Chat Models

In [6]:
chat_model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [33]:
messages = [
    SystemMessage(content="You're an assistant knowledgeable about the hospitals, patients, providers, physicians, and visits in a hospital system"),
    HumanMessage(content="How does pain treatment vary based on body location?"),
]

chat_model.invoke(messages)

AIMessage(content="Pain treatment can vary based on the location of the body for several reasons. The type and intensity of pain can differ depending on the body part affected, and the treatment approach may need to be tailored accordingly. Here are some examples of how pain treatment can vary based on body location:\n\n1. Musculoskeletal Pain: Pain in the muscles, bones, joints, or connective tissues often requires a combination of treatments. This may include physical therapy, stretching exercises, heat or cold therapy, massage, and nonsteroidal anti-inflammatory drugs (NSAIDs) for pain relief. In some cases, orthopedic interventions like joint injections or surgeries may be necessary.\n\n2. Head and Neck Pain: Headaches, migraines, and neck pain can be managed with various approaches. Over-the-counter pain relievers, muscle relaxants, and lifestyle modifications (such as stress reduction techniques) are commonly used. For more severe cases, prescription medications, nerve blocks, ph

In [34]:
chat_model.predict("How does pain treatment vary based on body location?")

"Pain treatment can vary based on body location due to several factors, including the underlying cause of pain, the type of pain experienced, and the specific treatment options available for that particular body part. Here are some examples of how pain treatment may vary based on body location:\n\n1. Back pain: Treatment for back pain can range from conservative measures such as physical therapy, stretching exercises, and pain medications, to more invasive options like epidural steroid injections or surgery, depending on the severity and cause of the pain.\n\n2. Joint pain: Joint pain, such as in the knees, hips, or shoulders, may be managed with a combination of nonsteroidal anti-inflammatory drugs (NSAIDs), physical therapy, joint injections (corticosteroids or hyaluronic acid), or in severe cases, joint replacement surgery.\n\n3. Headaches: Treatment for headaches can vary depending on the type and location of the headache. For example, tension headaches may be managed with over-the

#### Prompts

In [32]:
from langchain.prompts import ChatPromptTemplate

visit_template = """
                  You're an assistant knowledgeable about the hospitals, patients, providers, physicians, and visits in a hospital system.
                  Use the following description of visits to answer questions. Do not use any external knowledge besides what is provided 
                  in the visit description.
                  {descriptions}
                  
                  {question}
                  """
                  
review_template = """
                  You're an assistant that helps hospital admins understand how patients percieve ther hospitals by analyzing patient reviews.
                  Use the following reviews to answer questions:
                  {reviews}
                  
                  {question}
                  """

review_prompt = ChatPromptTemplate.from_template(review_template)
visit_prompt = ChatPromptTemplate.from_template(visit_template)
review_template.format(reviews="yeah boi", question="hello?")

"\n                  You're an assistant that helps hospital admins understand how patients percieve ther hospitals by analyzing patient reviews.\n                  Use the following reviews to answer questions:\n                  yeah boi\n                  \n                  hello?\n                  "

In [10]:
reviews = """
 "The medical staff at the hospital were incredibly attentive and supportive during my stay. The facilities were top-notch, making my recovery comfortable and smooth."
 "My time at the hospital was challenging. The nurses seemed overwhelmed, and communication between the staff and patients was lacking. Improvement is needed in patient care."
 "I'm grateful for the exceptional care I received at the hospital. The doctors were thorough in explaining my condition, and the compassionate nurses made my recovery a positive experience."
 "Unfortunately, my stay was marred by a lack of cleanliness in the facilities. It's essential for the hospital to maintain a hygienic environment to prevent infections and ensure patient well-being."
 "The hospital staff went above and beyond to make me feel comfortable and informed about my treatment. The positive attitude of the medical team greatly contributed to my overall well-being.
 "I had a mixed experience during my stay. While the medical care was satisfactory, the administrative process felt disorganized and resulted in unnecessary delays in my treatment."
 """
 
question = "Have any patients complained about a hospital's cleanliness? If so, describe the situation."

chain = review_prompt | chat_model
chain.invoke({"reviews":reviews, "question":question})

AIMessage(content="Yes, one patient has complained about a hospital's cleanliness. They mentioned that their stay was marred by a lack of cleanliness in the facilities. They emphasized the importance of maintaining a hygienic environment to prevent infections and ensure patient well-being.")

#### Retrieval

In [12]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings

loader = CSVLoader(file_path=REVIEWS_PATH, source_column="review")
reviews= loader.load()
print(reviews[0].page_content)

: 0
review_id: 0
visit_id: 6997
review: The medical staff at the hospital were incredibly attentive and supportive during my stay. The facilities were top-notch, making my recovery comfortable and smooth.


In [24]:
# Takes a while to run
embedding_model = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
reviews_vector_db = Chroma.from_documents(reviews, embedding_model, persist_directory="../src/chroma_data/review_embeddings/")

In [28]:
reviews_vector_db = Chroma(embedding_function=embedding_model, persist_directory="../src/chroma_data/review_embeddings/")

In [29]:
query = "Have any patients complained about a hospital being unclean? If so, describe the situations."
docs = reviews_vector_db.similarity_search(query)

for d in docs:
    
    print(d.page_content)

: 33
review_id: 33
visit_id: 7503
review: My stay at the hospital was marred by a lack of cleanliness in my room. It was evident that the facilities were not properly maintained. Disappointing given the importance of hygiene in a healthcare setting.
: 33
review_id: 33
visit_id: 7503
review: My stay at the hospital was marred by a lack of cleanliness in my room. It was evident that the facilities were not properly maintained. Disappointing given the importance of hygiene in a healthcare setting.
: 24
review_id: 24
visit_id: 7628
review: The cleanliness and hygiene standards at the hospital were exceptional, contributing to a sense of safety. However, the lack of family-friendly accommodations limited the support system available during my stay.
: 24
review_id: 24
visit_id: 7628
review: The cleanliness and hygiene standards at the hospital were exceptional, contributing to a sense of safety. However, the lack of family-friendly accommodations limited the support system available during m

In [37]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

review_vector_retriever = reviews_vector_db.as_retriever(k=10)

review_chain = (
    {"reviews": review_vector_retriever, "question": RunnablePassthrough()}
    | review_prompt
    | chat_model
    | StrOutputParser()
)

In [19]:
review_chain.invoke("Have any patients complained about a hospital being unclean? If so, describe the situations.")

'Yes, patients have complained about a hospital being unclean. In one review, the patient mentioned that their stay at the hospital was marred by a lack of cleanliness in their room. They observed that the facilities were not properly maintained, which was disappointing given the importance of hygiene in a healthcare setting.'

In [30]:
loader = CSVLoader(file_path=VISIT_DESCRIPTIONS_PATH, source_column="visit_description")
visit_descriptions = loader.load()
print(visit_descriptions[0].page_content)

: 0
visit_id: 0
visit_description: patient id: 0 
 date of admission: 2022-11-17 
 billing amount: 37490.98336352819 
 room number: 146 
 admission type: Elective 
 discharge date: 2022-12-01 
 test results: Inconclusive 
 visit id: 0 
 physician id: 102 
 payer id: 1 
 hospital id: 0 
 chief complaint: nan 
 treatment description: nan 
 primary diagnosis: nan 
 visit status: DISCHARGED 
 hospital name: Wallace-Hamilton 
 payer name: UnitedHealthcare 
 patient name: Tiffany Ramirez 
 patient sex: Female 
 patient dob: 1994-10-06 00:00:00 
 patient blood type: O+ 
 physician name: Leslie Williams 
 physician dob: 1990-08-16 00:00:00 
 physician grad year: 2020-08-16 00:00:00 
 medical school: University of Michigan Medical School 
 salary: 311158.79702275497


In [31]:
# Takes a while to run
embedding_model = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
visits_vector_db = Chroma.from_documents(visit_descriptions, embedding_model, persist_directory="../src/chroma_data/visit_embeddings/")

In [55]:
visits_vector_retriever = visits_vector_db.as_retriever(k=5)

visit_chain = (
    {"descriptions": visits_vector_retriever, "question": RunnablePassthrough()}
    | visit_prompt
    | chat_model
    | StrOutputParser()
)

#### Agents

In [35]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

In [78]:
def ask_about_reviews(query: str):
    
    return review_chain.invoke(query)

def ask_about_visits(query: str):
    
    return visit_chain.invoke(query)

tools = [
    Tool(
        name="Reviews",
        func=review_chain.invoke,
        description="""Useful when you need to answer questions about patient reviews or experiences at the hopsital.
        Not useful for answering questions about specific visit details such as payer, billing, treaetment, 
        diagnosis, chief complaint, hospital, or phyisican information. Pass the entire question as input to the tool.
        """,
    ),
    Tool(
        name="Visits",
        func=visit_chain.invoke,
        description="""Useful for answering questions about visit details such as payer, billing, treaetment, 
        diagnosis, chief complaint, hospital, or phyisican information. Only use the provided context to respond. 
        Pass the entire question as input to the tool.
        """,
    ),
]

hospital_rag_agent = initialize_agent(
    tools,
    ChatOpenAI(model="gpt-3.5-turbo", temperature=0),
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [72]:
hospital_rag_agent.run("Have any reviews expressed privacy concerns? Summarize them.")



> Entering new AgentExecutor chain...

Invoking: `Reviews` with `privacy concerns`


Based on the provided reviews, there are privacy concerns mentioned by patients. One patient mentioned having a challenging experience at the hospital due to the lack of privacy in shared rooms. They emphasized that privacy is essential for patients and improvements in this area would significantly enhance the overall patient experience. Another patient, however, commended the hospital's commitment to patient privacy. They mentioned that the staff ensured confidentiality, creating a safe space for open communication about health concerns.Yes, there have been reviews expressing privacy concerns. One patient mentioned having a challenging experience at the hospital due to the lack of privacy in shared rooms. They emphasized that privacy is essential for patients and improvements in this area would significantly enhance the overall patient experience. Another patient, however, commended the hospital's c

"Yes, there have been reviews expressing privacy concerns. One patient mentioned having a challenging experience at the hospital due to the lack of privacy in shared rooms. They emphasized that privacy is essential for patients and improvements in this area would significantly enhance the overall patient experience. Another patient, however, commended the hospital's commitment to patient privacy. They mentioned that the staff ensured confidentiality, creating a safe space for open communication about health concerns."

In [80]:
hospital_rag_agent.run("Have any reviews expressed privacy concerns? Summarize them.")



> Entering new AgentExecutor chain...
I should use the Reviews tool to search for reviews that mention privacy concerns.
Action: Reviews
Action Input: "privacy concerns"
Observation: Based on the provided reviews, there are privacy concerns mentioned by patients. One patient mentioned having a challenging experience at the hospital due to the lack of privacy in shared rooms. They emphasized that privacy is essential for patients and improvements in this area would significantly enhance the overall patient experience. Another patient, however, commended the hospital's commitment to patient privacy. They mentioned that the staff ensured confidentiality, creating a safe space for open communication about health concerns.
Thought:There are mixed reviews regarding privacy concerns at the hospital. Some patients have expressed concerns about the lack of privacy in shared rooms, while others have praised the hospital's commitment to patient privacy.
Final Answer: Yes, there are reviews that

'Yes, there are reviews that have expressed privacy concerns.'

In [79]:
hospital_rag_agent.run("List the full names of some physicians that have treated medicare patients")



> Entering new AgentExecutor chain...
I should use the Visits tool to find physicians who have treated Medicare patients.
Action: Visits
Action Input: "List physicians who have treated Medicare patients"
Observation: The physicians who have treated Medicare patients are:
1. Julie Walker
2. Ryan Gutierrez
3. Tamara Potter
4. Patrick Williams
Thought:I now know the final answer
Final Answer: The full names of some physicians that have treated Medicare patients are Julie Walker, Ryan Gutierrez, Tamara Potter, and Patrick Williams.

> Finished chain.


'The full names of some physicians that have treated Medicare patients are Julie Walker, Ryan Gutierrez, Tamara Potter, and Patrick Williams.'